In [39]:
from climada.util.api_client import Client
from climada.util.constants import RIVER_FLOOD_REGIONS_CSV
from climada_petals.entity.impact_funcs.river_flood import ImpfRiverFlood,flood_imp_func_set
import pandas as pd
from climada.engine import Impact
from climada.hazard import Hazard


In [44]:
def impact_calc_flood(country, regions_df, impf_set, scenario, year_range):
    client = Client()
    rf_hazard = client.get_hazard('river_flood', properties={'country_iso3alpha': country, 'climate_scenario': scenario, 'year_range':year_range})
    litpop = client.get_litpop_default(country=country)
    litpop.gdf['impf_RF'] = int(regions_df[regions_df['ISO']== country]['impf_RF'])
    litpop.gdf = litpop.gdf.drop(columns=['impf_'])
    #litpop.gdf = litpop.gdf[(litpop.gdf['latitude']<60)&(litpop.gdf['latitude']>-60)]
    rf_impact = Impact()
    rf_impact.calc(litpop,impf_set,rf_hazard, save_mat=True)
    return rf_impact


In [45]:
impf_set = flood_imp_func_set()
year_range = '1980_2000'
scenario = 'historical'
client = Client()
regions_df = pd.read_csv(RIVER_FLOOD_REGIONS_CSV)
countries = ['USA']
impact_dict_hist = {country: impact_calc_flood(country,regions_df, impf_set, scenario, year_range) for country in countries}



2022-02-03 07:45:58,810 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 129752 coordinates.


In [46]:
for country in countries:
    impact_dict_hist[country].write_sparse_csr("".join(["/Users/szelie/data/20211208_Multi-Impact_ZelieCarmen/river_flood_impact_150arcsec_historical_1980_2000_",country]))
    

In [47]:
for country in countries:
    impact_dict_hist[country].write_csv("".join(["/Users/szelie/data/20211208_Multi-Impact_ZelieCarmen/river_flood_impact_150arcsec_historical_1980_2000_",country, '.csv']))
    